#### <center> <img src="figures/logo.png" width="300"/>  </center>

# <center>  the ground deformation modelling   </center>


<p style="text-align: center;"><b><i>hands on very basic modelling for volcanoes</i></b></p>


<p style="text-align: center;"><b><i>by flavio cannavò </i></b> <a href= "mailto:flavio.cannavo@ingv.it">flavio.cannavo@ingv.it</a></p>

<p style="text-align: center;"><small>24 July 2023, Nicolosi, Italy</small></p>

 <center> <img src="figures/ingv.png" width="100"/>  </center>



<h1><strong>Spherical Source</strong>&nbsp;to model inflation and deflation causative body</h1>

We consider a spherical cavity of radius *a*, with center at depth *d* in a uniform half-space subjected to uniform internal pressure *p*.


The problem geometry.
<div> <img src="figures/Mogi1_.png" width="500"/> </div>


2 coordinate systems: 

 1. spherical with origin at the center of the spherical cavity (*R* radial distance)
 2. cylindrical centered above the chamber (*r* radial distance from the center at the surface)

Boundary conditions:
$$
\begin{aligned}
\sigma_{i3}=0  && on && x_3=0 \\
\sigma_{rr}=-p  && on && r=a  \\
\sigma_{r\theta}=\sigma_{r\phi}=0  && on && r=a \\
\end{aligned}
$$

*(i.e. traction free at the surface; radial stress uniform pressure on chamber walls where the shear tractions vanish)*



Boundary conditions:
$$
\begin{aligned}
\sigma_{i3}=0  && on && x_3=0 \\
\sigma_{rr}=-p  && on && r=a  \\
\sigma_{r\theta}=\sigma_{r\phi}=0  && on && r=a \\
\end{aligned}
$$

<p>The first step is to solve the problem in a <strong>full-space</strong>, with <em>radial symmetry</em>, when the stress equilibrium reduces to:</p>

$$
\frac{\partial \sigma_{rr}}{\partial r}+\frac{1}{r}(2\sigma _{rr}-\sigma_{\theta\theta}-\sigma_{\phi\phi}) = 0
$$


<p style="text-align: center;"><span style="color:#e74c3c">... developing in a few simple steps ... </span></p>



<p style="text-align: center;"><span style="color:#e74c3c"> Kidding! The solution is far from trivial ... for another kind of class ... </span></p>

For a point source approximation meaning $ a \ll d $ (in practice, approximation is good for $a < 0.5d$), we obtain the predicted deformation field at the surface (<span style="color:#3498db">Mogi, 1958</span>):

$$
\begin{aligned}
u_r = \frac{(1-\nu)\Delta P a^3}{G} \frac{r}{(r^2+d^2)^{3/2}} \\
u_z = \frac{(1-\nu)\Delta P a^3}{G} \frac{d}{(r^2+d^2)^{3/2}} \\
\end{aligned}
$$

where ΔP is source overpressure (*p*), ν Poisson's ratio and G shear modulus.

The volume change ΔV is given by the integral of the radial displacement over the surface of the cavity. If the cavity radius is small compared to the depth of burial it can be approximated as:

$$
\Delta V \approx \pi \Delta P a^3 /G
$$

Thus the another formulation is:

$$
\begin{aligned}
u_r = \frac{(1-\nu)\Delta V}{\pi} \frac{r}{(r^2+d^2)^{3/2}} \\
u_z = \frac{(1-\nu)\Delta V}{\pi} \frac{d}{(r^2+d^2)^{3/2}} \\
\end{aligned}
$$




🗒<span style="color:#4e5f70">Notes: 
    <ol>
	<li>the deformation is proportional to the volume/pressure change;</li>
	<li>magnitude of (3D) displacement vector decreases&nbsp; as 1/distance from source</li>
	<li>there is an ambiguity between &Delta;P and *a* that does not allow them to be derived independently;</li>
	<li>&Delta;V cannot exactly equate to change in mass because of compressibility of magma.</li>
</ol></span>

# learning by doing

Let's start by importing standard python libraries

In [ ]:
# import standard libraries for computation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
np.random.seed(240723)

# Set notebook mode to work in offline
import plotly.offline as pyo
import plotly.io as pio
pyo.init_notebook_mode()
pio.renderers.default = 'notebook'
import plotly.graph_objects as go

from scipy.optimize import minimize, direct
from collections import OrderedDict
from improve import plot3Dfield

Let's implement the point source reformulating it in a Cartesian coordinate system 

<p style="text-align: center;"><i>(x=easting, y=northing, d=depth positive downwards):</i></p>


<table align="center" border="0" cellpadding="1" cellspacing="1" >
	<tbody>
		<tr>
			<td>
$$
\begin{aligned}
u_x = \frac{(1-\nu)\Delta V}{\pi} \frac{x}{(x^2+y^2+d^2)^{3/2}} \\
u_y = \frac{(1-\nu)\Delta V}{\pi} \frac{y}{(x^2+y^2+d^2)^{3/2}} \\
u_z = \frac{(1-\nu)\Delta V}{\pi} \frac{d}{(x^2+y^2+d^2)^{3/2}} \\
\end{aligned}
$$</td>
			<td> 
            &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
            </td>
			<td>
$$
\begin{aligned}
u_x = \frac{(1-\nu)\Delta P a^3}{G} \frac{x}{(x^2+y^2+d^2)^{3/2}} \\
u_y = \frac{(1-\nu)\Delta P a^3}{G} \frac{y}{(x^2+y^2+d^2)^{3/2}} \\
u_z = \frac{(1-\nu)\Delta P a^3}{G} \frac{d}{(x^2+y^2+d^2)^{3/2}} \\
\end{aligned}
$$</td>
		</tr>
	</tbody>
</table>




In [ ]:
def pointsource(x, y, xs, ys, d, dV=None, nu=0.25, G=30e9, a=None, dP=None):
    """
    Calculates surface deformation based on point source
    References: Mogi 1958
    Implement both formulations: dV or dP

    Args:
    ------------------
    x: x-coordinate grid (m)
    y: y-coordinate grid (m)
    xs: x point source epicenter (m)
    ys: y point source epicenter (m)
    d: depth to point (m)
    
    dV: change in volume (m^3)
    a: source radius (m) must be << d
    G: shear modulus (Pa)
    dP: excess pressure of the source (Pa)
        nu: poisson's ratio for medium
    
    Returns:
    -------
    deformation at surface (ux, uy, uz)
    """
    
    # Center coordinate grid on point source
    dx = x - xs
    dy = y - ys
    dz = d
    
    R2 = dx*dx + dy*dy + dz*dz
    
    # choose the formulation based on the valid parameter dP or dV
    if dP is None and dV is not None:
            c = ((1-nu) / np.pi) * dV            
    elif dP is not None and dV is None:
            c = a*a*a*dP*(1-nu)/G            
    else:
            c = None # ambiguous parameters
                        
    C = c / pow(R2, 1.5)
    
    return np.array([C*dx,C*dy,C*dz])

<p>Now that we have a <u>mathematical model </u>that can describe a <u>ground deformation field </u>given some <u>source parameters,</u> <strong>let&#39;s play with it!</strong></p>

In [ ]:
# set model parameters (4 for a volumetric point source), located about @Etna

xs = 495000      #m UTM horizontal location
ys = 4180000     #m UTM horizontal location
d = 2000         #m depth b.s.l.
dV = 3e6         #m^3 

# create a range of coordinates at step 100 m around the source where to probe the deformation field
edgesemilength = 8000
stepgrid = 100 # m
xsteps = np.linspace(xs-edgesemilength, xs+edgesemilength, int(2*edgesemilength/stepgrid))
ysteps = np.linspace(ys-edgesemilength, ys+edgesemilength, int(2*edgesemilength/stepgrid))




Use the direct model of point source to calculate the predicted deformation field


In [ ]:
# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

# set some source parameters (PLAY WITH THEM!!)
params = OrderedDict([('xs',xs),
            ('ys',ys),
            ('d', 1000), #m
            ('dV', 3e6), #m^3
            ])

# call the direct model of point source to calculate the predicted deformation at the probing points
ux,uy,uz = pointsource(eastings, northings, **params)

# Create four axes and access them through the returned array
fig, axs = plt.subplots(2, 2, figsize=(13, 11))

# east component
c = axs[0, 0].pcolormesh(eastings/1000, northings/1000, ux, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 0].set_title('x-component')
clb = fig.colorbar(c, ax=axs[0, 0])
clb.ax.set_title('m')

# north component
c = axs[0, 1].pcolormesh(eastings/1000, northings/1000, uy, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 1].set_title('y-component')
clb = fig.colorbar(c, ax=axs[0, 1])
clb.ax.set_title('m')

# vertical component
c = axs[1, 0].pcolormesh(eastings/1000, northings/1000, uz, cmap='inferno', vmin=np.min(uz), vmax=np.max(uz))
axs[1, 0].set_title('z-component')
clb = fig.colorbar(c, ax=axs[1, 0])
clb.ax.set_title('m')

# total deformation
c = axs[1, 1].pcolormesh(eastings/1000, northings/1000, np.sqrt(ux**2 + uy**2 + uz**2), cmap='inferno', 
                                                                            vmin=np.min(uz), vmax=np.max(uz))
axs[1, 1].set_title('total')
clb = fig.colorbar(c, ax=axs[1, 1])
clb.ax.set_title('m')

plt.show()

Now let us take a closer look at the predicted deformation along a line

In [ ]:
# create the list of source parameters
params = OrderedDict([('xs',xs),
            ('ys',ys),
            ('d', d), #m
            ('dV', dV), #m^3
            ])

# call the direct model of point source to calculate the predicted deformation at the probing points
ux,uy,uz = pointsource(xsteps, ys, **params)

# plot results
fig = go.Figure()

# source position
fig.add_vline( x = xs)

fig.add_trace(go.Scatter(x=xsteps, y=ux, name="dx [East]" ))

fig.add_trace(go.Scatter(x=xsteps, y=uy, name="dy [North]" ))

fig.add_trace(go.Scatter(x=xsteps, y=uz, name="dz [Vertical]" ))

fig.update_layout(
    title="Mogi Point Source Predicted Displacement",
    xaxis_title="Easting [m]",
    yaxis_title="displacement [m]",
    legend_title="component",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

Now let's try to see how the deformation field changes as the depth changes.

In [ ]:
# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

# iterate for different depths
for d in [500, 1000, 2000, 5000]:
    params = OrderedDict([('xs',xs),
                ('ys',ys),
                ('d', d), #m
                ('dV', 3e6), #m^3
                ])

    # call the direct model of point source to calculate the predicted deformation at the probing points
    ux,uy,uz = pointsource(eastings, northings, **params)

    # Create four polar axes and access them through the returned array
    fig, axs = plt.subplots(2, 2, figsize=(9, 8))

    fig.suptitle(f'depth {d} m', fontsize=16)
    # east component
    c = axs[0, 0].pcolormesh(eastings/1000, northings/1000, ux, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
    axs[0, 0].set_title('x-component')
    clb = fig.colorbar(c, ax=axs[0, 0])
    clb.ax.set_title('m')

    # north component
    c = axs[0, 1].pcolormesh(eastings/1000, northings/1000, uy, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
    axs[0, 1].set_title('y-component')
    clb = fig.colorbar(c, ax=axs[0, 1])
    clb.ax.set_title('m')

    # vertical component
    c = axs[1, 0].pcolormesh(eastings/1000, northings/1000, uz, cmap='viridis', vmin=np.min(uz), vmax=np.max(uz))
    axs[1, 0].set_title('z-component')
    clb = fig.colorbar(c, ax=axs[1, 0])
    clb.ax.set_title('m')

    # total deformation
    c = axs[1, 1].pcolormesh(eastings/1000, northings/1000, np.sqrt(ux**2 + uy**2 + uz**2), cmap='viridis', 
                                                                                vmin=np.min(uz), vmax=np.max(uz))
    axs[1, 1].set_title('total')
    clb = fig.colorbar(c, ax=axs[1, 1])
    clb.ax.set_title('m')

    plt.show()

## Violating the  $ a \ll d $ assumption!


We have said that the model is an approximation and the assumption we have made is to assume a depth much greater than the radius of the sphere.

Now let us try to violate this assumption and we will see that the deformation field is entirely realistic. So can we accept it? 
You will have the answer in the afternoon.





In [ ]:
# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

# now use the point source versione with dP and the formula that links dP to dV
params = OrderedDict([('xs',xs),
            ('ys',ys),
            ('d', 500), #m
            ('a', 500), #m
                   #dV   G           a  
            ('dP', 3e6*3e10/(np.pi*(500**3)))
            ])

# call the direct model of point source to calculate the predicted deformation at the probing points
ux,uy,uz = pointsource(eastings, northings, **params)

# Create four polar axes and access them through the returned array
fig, axs = plt.subplots(2, 2, figsize=(13, 11))

# east component
c = axs[0, 0].pcolormesh(eastings/1000, northings/1000, ux, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 0].set_title('x-component')
clb = fig.colorbar(c, ax=axs[0, 0])
clb.ax.set_title('m')

# north component
c = axs[0, 1].pcolormesh(eastings/1000, northings/1000, uy, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 1].set_title('y-component')
clb = fig.colorbar(c, ax=axs[0, 1])
clb.ax.set_title('m')

# vertical component
c = axs[1, 0].pcolormesh(eastings/1000, northings/1000, uz, cmap='viridis', vmin=np.min(uz), vmax=np.max(uz))
axs[1, 0].set_title('z-component')
clb = fig.colorbar(c, ax=axs[1, 0])
clb.ax.set_title('m')

# total deformation
c = axs[1, 1].pcolormesh(eastings/1000, northings/1000, np.sqrt(ux**2 + uy**2 + uz**2), cmap='viridis', 
                                                                            vmin=np.min(uz), vmax=np.max(uz))
axs[1, 1].set_title('total')
clb = fig.colorbar(c, ax=axs[1, 1])
clb.ax.set_title('m')

plt.show()

# Deformation field seen by GNSS sensors

We have seen that a deformation field can be measured using GNSS sensors that return the 3 components of the field. East-West, North-South and vertical deformation.

What you see with a GNSS sensor permanently installed at a point is a time series of the position, from which displacements can be derived (in the simplest manner as the difference between two time instants).

#### <center> <img src="figures/GPSexample.jpg" width="500"/>  </center>




The representation of these measurements is a so-called displacement vector indicating the movement of the measurement point.

Let's plot a deformation field as seen by GNSS sensors.

In [ ]:
# create a range of coordinates at step 2000 m
stepgrid = 2000 # m
edgesemilength = 8000
xsteps = np.linspace(xs-edgesemilength, xs+edgesemilength, int(2*edgesemilength/stepgrid))
ysteps = np.linspace(ys-edgesemilength, ys+edgesemilength, int(2*edgesemilength/stepgrid))

# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

# calculate the predicted deformation components
ux,uy,uz = pointsource(eastings, northings, **params)

Plot the 3 components of the deformation field in a 3-D space, sampled in the points of a mesh grid

In [ ]:
# call a function to plot a vectorial field in 3D using interactive plotly library
fig = plot3Dfield(ux, uy, uz, eastings, northings, arrow_tip_ratio = 0.4,
                arrow_starting_ratio = 0.90, scalevertfactor = 30000)

print(f'Maximum vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# Add the point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='source'))



camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5)
)

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field'
)

fig.show()

# Topography correction

The interaction of a source with the irregular surface is significant only when the burial depth is of the same order as the characteristic horizontal scale of the topography.

The planar free surface of an elastic half-space is a poor match to the high relief of volcanic regions.

**What distortions are introduced by topography to the deformation from a buried magmatic source?**

Topographic relief imparts a slope to the surface and it alters the distance from the surface to the source.  
*We have shown that surface deformation varies with some power of the inverse distance from the source*.

<div> <center> <img src="figures/topocorr.png" width="400"/> </center> </div>

<p>&nbsp;1. The simplest topographic correction consists of an offset of the free surface by a constant amount, which is called the <u><strong>reference elevation</strong></u>. Without constructing a numerical model to test for the optimal reference elevation, <span style="color:#2980b9">Williams and Wadge</span> (<span style="color:#2980b9">2000</span>) provide the following guidelines: use a reference elevation somewhere between the <u><strong>average</strong></u> and the maximum for shallow sources and decrease this value with increasing source depth.<br />
&nbsp;</p>

<p>&nbsp;2. The <u><strong>varying depth model </strong></u>corrects for the changing distance between the source and the surface by varying the magma chamber depth with topography.</p>

<p><br />
&nbsp;3. <span style="color:#2980b9">Williams and Wadge</span> (<span style="color:#2980b9">2000</span>) developed the <u><strong>topographically corrected method </strong></u>to include arbitrary topography in elastic half-space source models. The application of this method is complex, requiring free-surface displacements, derivatives, and curvatures for the elastic half-space source model and a decimated DEM.</p>

Implement the point source considering the **varying-depth topographic corretion**, thus including the station elevation.
For each measurement point the depth of the source will be different.

In [ ]:
    def pointsource3D(x, y, z, xs, ys, d, dV=None, nu=0.25, G=30e9, a=None, dP=None):
        
        """
        Calculates surface deformation based on point source with Varying-Depth topography correction
        
        References: Mogi 1958, Williams and Wadge 2000

        Args:
        ------------------
        x: x-coordinate grid (m)
        y: y-coordinate grid (m)
        z: z-coordinate grid (m)
        xs: x point source epicenter (m)
        ys: y point source epicenter (m)
        d: depth to point (m)

        dV: change in volume (m^3)

        nu: poisson's ratio for medium

        Returns:
        -------
        deformation at surface u = [ux, uy, uz]
        """
        
         # relative points of measurement
            
        repoints = np.array([x.reshape(-1), y.reshape(-1), z.reshape(-1)]).T - np.array([xs, ys, -d]) 
                       
        # calculate the distances
        R = np.linalg.norm(repoints,axis=1)
        
        if dP is None and dV is not None:
            C = ((1-nu) / np.pi) * dV           
        elif dP is not None and dV is None:
            C = a*a*a*dP*(1-nu)/G
        else:
            C = None # ambiguous parameters
                        
        # Mogi solution
        u = np.array([C*p/(Ri**3) for p, Ri in zip(repoints,R)])
                    
        return u[:,0].reshape(x.shape), u[:,1].reshape(x.shape), u[:,2].reshape(x.shape)


Now, let's consider a fictitious topography, bell-shaped imitating a volcano pattern

In [ ]:
# define a bell function that mimics a volcano topography
def belltopo(x,y, x0,y0, sigma):
    return np.exp(-((x-x0)**2 + (y-y0)**2)/(sigma**2))   
    
elevations = belltopo(eastings, northings, xs, ys, 6000)*3300 #m

In [ ]:
# calculate the deformation field
ux,uy,uz = pointsource3D(eastings, northings, elevations, **params)

Plot the field and check the 3-dimensionality of the points

In [ ]:
fig = plot3Dfield(ux, uy, uz, eastings, northings, elevations, arrow_tip_ratio = 0.7,
                arrow_starting_ratio = 0.90)

print(f'Maximum vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# Add the point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='source'))

# mesh to plot the topography
topox, topoy = np.meshgrid(np.linspace(xsteps[0], xsteps[-1], 200), np.linspace(ysteps[0], ysteps[-1], 200))
topoz = belltopo(topox, topoy, xs, ys, 6000)*3300

# add topography
fig.add_trace(go.Surface(x=topox, y=topoy, z=topoz, opacity = .7, showscale = False, 
                         colorscale = 'Greys_r',
                         name = "Topography"))



camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field with Topography')

fig.show()

# <center>  Geophysical inverse problems </center> 

## <center> Inverse problems = inference about physical systems from data </center> 

<div> <center><img src="figures/theil.png" width="400"/> </center></div>

<p>Issues:</p>
<ul>
<li>Data usually contain errors (<strong>data uncertainties</strong>)</li>
<li>We always have a <strong>finite amount of data</strong></li>
<li>Physical <strong>theories</strong> require <strong>approximations</strong></li>
<li>Our physical theory may be inaccurate (<strong>theoretical uncertainties</strong>)</li>
<li>Our forward problem may be <strong>highly nonlinear</strong></li>
<li>Infinitely many models will fit the data (<strong>non-uniqueness</strong>)</li>
</ul>
<p></p>
<p style="padding-left: 40px; text-align: center;"></p>
<p style="padding-left: 40px; text-align: center;">Three classical questions (from <span style="color: #0000ff;">Backus and Gilbert, 1968</span>)</p>
<ol>
<li style="list-style-type: none;">
<ol>
<li style="list-style-type: none;">
<ol>
<li>The problem of the <span style="color: #339966;"><strong>existence</strong></span>: does any model fit the data ?</li>
<li>The problem of <span style="color: #339966;"><strong>uniqueness</strong></span>: is there a unique model that fits the data ?</li>
<li>The <strong><span style="color: #339966;">stability</span></strong> problem: can small changes in the data produce large changes in the solution ?</li>
</ol>
</li>
</ol>
</li>
</ol>
<p style="padding-left: 40px; text-align: center;">otherwise ... <span style="color: #ff0000;"><strong>ill-posed problem</strong></span>! ... (<em>as our problem is</em>)</p>

<p><em>Mathematically:&nbsp; &nbsp;</em><strong>Inversion = Optimization of a fitness function in the space of parameters</strong></p>
<p><img src="figures/optims.png" alt="Optimization" style="float: right;" width="500"/></p>
<p>Typically minimization of <span style="text-decoration: underline;">misfit between data &amp; model output</span></p>
<p>Most of the cases non-linear and non-covex optimization (so many local minima)</p>
<p>An inversion procedure/optimization should be carried out with <em>bounded parameters</em></p>
<div id="gtx-trans" style="position: absolute; left: 512px; top: 69px;">
<div class="gtx-trans-icon"></div>
</div>


$$ wrmse = \sqrt{\sum_{data}\left[ \frac{(m_i-p_i)^2}{e_i^2} \right]} $$

In [ ]:
# let's define the misfit function
def wrmse(modelparams, x, y, z, ux, uy, uz, eux=1., euy=1., euz=1., nu=0.25): 
        
    xs, ys, d, dV = modelparams
    
    ux_, uy_, uz_ = pointsource3D(x, y, z, xs, ys, d, dV, nu)

    return np.sqrt((((ux_ - ux)/eux) ** 2 + ((uy_ - uy)/euy) ** 2 + ((uz_ - uz)/euz) ** 2).mean())

We now create the data for a synthetic deformation field from a known source and then try to find the pattern from the data by inversion.

<p></p>
<div> <center> <img src="figures/dirinv.gif" width="400"/> </center> </div>



In [ ]:
# create the dataset of 3D displacements due to a point source

xs = 500000      #m UTM horizontal location
ys = 4177000     #m UTM horizontal location
d = 1500         #m depth b.s.l.
dV = 3e6         #m^3 

# create a range of coordinates at step 100 m
edgesemilength = 8000
stepgrid = 2000 # m
xsteps = np.linspace(xs-edgesemilength, xs+edgesemilength, int(2*edgesemilength/stepgrid))
ysteps = np.linspace(ys-edgesemilength, ys+edgesemilength, int(2*edgesemilength/stepgrid))

# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

# consider the topography
elevations = belltopo(eastings, northings, xs, ys, 6000)*3300 #m

# calculate the deformation data
ux,uy,uz = pointsource3D(eastings, northings, elevations, xs=xs, ys=ys, d=d, dV=dV)

<p>With only the data obtained, we now try to derive the source parameters (whose nominal values we know).</p>
<p>To do the inversion, we can use a multi-step algorithm:</p>
<p style="padding-left: 40px;">- we give an <strong>assumed value (our best guess)</strong> to the parameters from which to start the search<br />- with a <strong>heuristic global optimization</strong> algorithm we obtain other parameter values that are candidates for the optimal solution</p>
<p style="padding-left: 40px;">We apply a <strong>direct search</strong> optimization algorithm to each of these two potential solutions.</p>
<p style="padding-left: 40px;">We choose the solution found <em>best in terms of misfit!</em></p>
<p></p>
<p></p>

<p>As for the heuristic global optimization algorithm we can use for example the DIRECT: DIviding RECTangles is a deterministic global optimization algorithm capable of minimizing a black box function with its variables subject to lower and upper bound constraints by sampling potential solutions in the search space (<span style="color: #3366ff;">Jones, Perttunen &amp; Stuckman, 1993</span>).</p>

<p>As for the direct search algorithm we can use for example the Nelder-Mead method (<span style="color: #3366ff;">Nelder and Mead, 1965</span>).</p>

In [ ]:
# Inversion
# ----------------

# define the misfit function of the model parameters
errfunc = lambda modelparams: wrmse(modelparams, eastings, northings, elevations, 
                                                             ux, uy, uz, nu=0.25)

# define the boundary conditions
#          xs[m]      ys[m]    d [m]   dV [m^3]
lwbound = [xs-10000, ys-10000,  0,    -1e7 ]        
upbound = [xs+10000, ys+10000, 6000,  1e7]

# define our best guess of the parameter values
initial_guess = [xs+1000,  ys-1000,     3000,   1e6]

# Global heuristic optimization, 2-steps: global search + direct search

# DIRECT global constrained optimization
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

# take the optimal parameters found
optimal_xs, optimal_ys, optimal_d, optimal_dV = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

# calculate the predicted deformation field with the found parameters
ux_, uy_, uz_ = pointsource3D(eastings, northings, elevations, 
                              optimal_xs, optimal_ys, optimal_d, optimal_dV)

# create a table to compare the parameters with the nominal ones
comparison = pd.DataFrame([[xs, ys, d, dV], [optimal_xs, optimal_ys, optimal_d, optimal_dV]], 
                          columns=['easting [m]','northing [m]','depth [m]','dV [m^3]'])
comparison.index = ['real','retrieved']

Plot the deformation field predicted by the optimal model vs the real field and compare the parameters with the nominal ones.

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(ux, uy, uz, eastings, northings, elevations, color='black', arrow_tip_ratio = 0.7,
                arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='black',
                    name='real source'))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, eastings, northings, elevations, fig=fig, color='red', arrow_tip_ratio = 1.0,
                arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# Add the found point source
fig.add_trace(go.Scatter3d(x=[optimal_xs], y=[optimal_ys], z=[-optimal_d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='estimated source'))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field')

display(comparison)

fig.show()

<p style="text-align: center;"><center><strong><img src="figures/smiley-cool.gif" alt="cool" /> Easy Peasy!</strong></center></p>
<p><span>To be more realistic our data should be affected by noise.</span></p>
<p><span>So let's add noise to our synthetic measurements!</span></p>

In [ ]:
sigmahorz = 0.01 #m
sigmavert = 0.02 #m

# additive white Gaussian noise
noisy_ux = ux + np.random.normal(0, sigmahorz, ux.shape)
noisy_uy = uy + np.random.normal(0, sigmahorz, uy.shape)
noisy_uz = uz + np.random.normal(0, sigmavert, uz.shape)

and now do inversion on the **noisy data**

In [ ]:
# Inversion
# ----------------

errfunc = lambda modelparams: wrmse(modelparams, eastings, northings, elevations, 
                                                             noisy_ux, noisy_uy, noisy_uz, nu=0.25)

#          xs[m]      ys[m]    d [m]   dV [m^3]
lwbound = [xs-10000, ys-10000,  0,    -1e7 ]        
upbound = [xs+10000, ys+10000, 6000,  1e7]

initial_guess = [xs+1000,  ys-1000,     3000,   1e6]

# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_d, optimal_dV = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

ux_, uy_, uz_ = pointsource3D(eastings, northings, elevations, 
                              optimal_xs, optimal_ys, optimal_d, optimal_dV)


comparison = pd.DataFrame([[xs, ys, d, dV], [optimal_xs, optimal_ys, optimal_d, optimal_dV]], 
                          columns=['easting [m]','northing [m]','depth [m]','dV [m^3]'])
comparison.index = ['real','retrieved']

Plot the deformation field predicted by the optimal model vs the real field and compare the parameters with the nominal ones.

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(noisy_ux, noisy_uy, noisy_uz, eastings, northings, elevations, color='black', 
                  arrow_tip_ratio = 0.3,
                  arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='black',
                    name='real source'))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, eastings, northings, elevations, fig=fig, color='red', arrow_tip_ratio = 0.8,
                arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[optimal_xs], y=[optimal_ys], z=[-optimal_d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='estimated source'))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5)
)

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field'
)

display(comparison)

fig.show()

We generally do not have many measuring stations, and probably not even arranged in a regular manner

So let's go with an even more realistic, less stations ... (soon you will see that there are other instruments able to mitigate this problem)

In [ ]:
# define how many stations we want
nstations = 10

# pick randomly some stations
idx = random.sample(list(range(np.size(noisy_ux))), nstations)
few_noisy_ux = noisy_ux.ravel()[idx]
few_noisy_uy = noisy_uy.ravel()[idx]
few_noisy_uz = noisy_uz.ravel()[idx]
few_eastings = eastings.ravel()[idx]
few_northings = northings.ravel()[idx]
few_elevations = elevations.ravel()[idx]

Re-try the inversion with a few noisy data

In [ ]:
# Inversion
# ----------------

errfunc = lambda modelparams: wrmse(modelparams, few_eastings, few_northings, few_elevations, 
                                                             few_noisy_ux, few_noisy_uy, few_noisy_uz, nu=0.25)

#          xs[m]      ys[m]    d [m]   dV [m^3]
lwbound = [xs-10000, ys-10000,  100,    -1e7 ]        
upbound = [xs+10000, ys+10000, 5000,  1e7]

initial_guess = [xs+1000,  ys-1000,     3000,   1e6]

# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_d, optimal_dV = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

ux_, uy_, uz_ = pointsource3D(few_eastings, few_northings, few_elevations, 
                              optimal_xs, optimal_ys, optimal_d, optimal_dV)


comparison = pd.DataFrame([[xs, ys, d, dV], [optimal_xs, optimal_ys, optimal_d, optimal_dV]], 
                          columns=['easting [m]','northing [m]','depth [m]','dV [m^3]'])
comparison.index = ['real','retrieved']


Plot the results and compare the solution

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(few_noisy_ux, few_noisy_uy, few_noisy_uz, few_eastings, few_northings, few_elevations, 
                  color='black', arrow_tip_ratio = 0.7,
                arrow_starting_ratio = 0.90)

print('Maximum synthetic vector length '+
      f'{np.max(np.linalg.norm([few_noisy_ux.ravel(), few_noisy_uy.ravel(), few_noisy_uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='black',
                    name='real source'))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, few_eastings, few_northings, few_elevations, fig=fig, color='red', 
                  arrow_tip_ratio = 0.7, arrow_starting_ratio = 0.90)


print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[optimal_xs], y=[optimal_ys], z=[-optimal_d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='estimated source'))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5)
)

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field'
)

display(comparison)

fig.show()

<p><center><img src="figures/uncertainty.jpeg" alt="uncert" width="400" /></center></p>


We haven't considered yet the measurement associated uncertainties!!!

So let's associate uncertanties to our noisy data by taking the deviation from the nominal values.

In [ ]:
# create false uncertainties but realistic

eux = np.abs((few_noisy_ux - ux.ravel()[idx]))
euy = np.abs((few_noisy_uy - uy.ravel()[idx]))
euz = np.abs((few_noisy_uz - uz.ravel()[idx]))

Re-try the inversion considering the data associated uncertainties.

$$ wrmse = \sqrt{\sum_{data}\left[ \frac{(m_i-p_i)^2}{e_i^2} \right]} $$

In [ ]:
# Inversion
# ----------------

errfunc = lambda modelparams: wrmse(modelparams, few_eastings, few_northings, few_elevations, 
                                                             few_noisy_ux, few_noisy_uy, few_noisy_uz, 
                                                            eux=eux, euy=euy, euz=euz, nu=0.25)

#          xs[m]      ys[m]    d [m]   dV [m^3]
lwbound = [xs-10000, ys-10000,  0,    -1e7 ]        
upbound = [xs+10000, ys+10000, 6000,  1e7]

initial_guess = [xs+1000,  ys-1000,     3000,   1e6]

# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_d, optimal_dV = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

ux_, uy_, uz_ = pointsource3D(few_eastings, few_northings, few_elevations, 
                              optimal_xs, optimal_ys, optimal_d, optimal_dV)


comparison = pd.DataFrame([[xs, ys, d, dV], [optimal_xs, optimal_ys, optimal_d, optimal_dV]], 
                          columns=['easting [m]','northing [m]','depth [m]','dV [m^3]'])
comparison.index = ['real','retrieved']


Plot the results and compare the solution

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(few_noisy_ux, few_noisy_uy, few_noisy_uz, few_eastings, few_northings, few_elevations, 
                  color='black', arrow_tip_ratio = 0.7,
                arrow_starting_ratio = 0.90)

print('Maximum synthetic vector length '+
      f'{np.max(np.linalg.norm([few_noisy_ux.ravel(), few_noisy_uy.ravel(), few_noisy_uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[xs], y=[ys], z=[-d],
                    mode='markers',
                    marker_size=14,
                    marker_color='black',
                    name='real source'))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, few_eastings, few_northings, few_elevations, fig=fig, color='red', 
                  arrow_tip_ratio = 0.7, arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Scatter3d(x=[optimal_xs], y=[optimal_ys], z=[-optimal_d],
                    mode='markers',
                    marker_size=14,
                    marker_color='red',
                    name='estimated source'))


camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5)
)

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field'
)

display(comparison)

fig.show()

# Modelling a dyke, the Okada analytical model

<p>The Okada model, named after its creator, Yoshimitsu Okada, is a fundamental and widely used method for <span style="text-decoration: underline;">modeling ground deformation</span> <span style="text-decoration: underline;">caused by seismic and volcanic events</span>.</p>
<p>Developed in the late 1980s, the Okada model provides a mathematical framework to understand and predict surface displacements resulting from the movement of <strong>faults</strong> or <strong>magma</strong> beneath the Earth's crust.</p>
<p></p>
<p><center><img src="figures/okada.jpg" alt="Okada" width=400/></center></p>
<p></p>
<p><span>It takes into account parameters such as fault length, width, depth of the top edge, orientations and the amount of slip or tensile that occurs during the event.</span></p>
<p><span>Dykes, usually, are characterized mainly with tensile components.</span></p>

<p>The equations and implementation are much more complex and longer than the Mogi source, so 
    <strong>let's avoid them !!!<img src="figures/smiley-wink.gif" alt="wink" /></strong></p>

In [ ]:
from improve import okada, plotOkada

# set model parameters

xs = 495000      #m UTM horizontal location
ys = 4180000     #m UTM horizontal location
zs = -2000         #m elevation a.s.l. of the middle top point of the rectangle
length = 5000  #m of the top
width = 4000   #m
azimuth = 187   # ° degrees from North
dip = 88       # ° degrees, 90 vertical
opening = 3  # m

params = OrderedDict([('xs',xs),
            ('ys',ys),
            ('zs', zs), #m
            ('length', length), #m
            ('width', width), #m
            ('azimuth', azimuth), #m
            ('dip', dip), #m
            ('opening', opening), #m
            ])


# create a range of coordinates at step 100 m to see the deformation field
edgesemilength = 10000
stepgrid = 100 # m
xsteps = np.linspace(xs-edgesemilength, xs+edgesemilength, int(2*edgesemilength/stepgrid))
ysteps = np.linspace(ys-edgesemilength, ys+edgesemilength, int(2*edgesemilength/stepgrid))

Calculate and plot the predicted deformation field by the Okada model given the parameter values.

You will notice that the field is more complex and not centrally symmetrical.

In [ ]:
# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)

elevations = eastings*0

ux,uy,uz = okada(eastings, northings, elevations, **params)

# Create four polar axes and access them through the returned array
fig, axs = plt.subplots(2, 2, figsize=(13, 11))

# east component
c = axs[0, 0].pcolormesh(eastings/1000, northings/1000, ux, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 0].set_title('x-component')
clb = fig.colorbar(c, ax=axs[0, 0])
clb.ax.set_title('m')

# north component
c = axs[0, 1].pcolormesh(eastings/1000, northings/1000, uy, cmap='RdBu_r', vmin=np.min(ux), vmax=np.max(ux))
axs[0, 1].set_title('y-component')
clb = fig.colorbar(c, ax=axs[0, 1])
clb.ax.set_title('m')

# vertical component
c = axs[1, 0].pcolormesh(eastings/1000, northings/1000, uz, cmap='viridis', vmin=np.min(uz), vmax=np.max(uz))
axs[1, 0].set_title('z-component')
clb = fig.colorbar(c, ax=axs[1, 0])
clb.ax.set_title('m')

# total deformation
c = axs[1, 1].pcolormesh(eastings/1000, northings/1000, np.sqrt(ux**2 + uy**2 + uz**2), cmap='viridis', 
                                                                            vmin=np.min(uz), vmax=np.max(uz))
axs[1, 1].set_title('total')
clb = fig.colorbar(c, ax=axs[1, 1])
clb.ax.set_title('m')

plt.show()

Let's plot the deformation vector as seen by GNSS stations on a regular grid.

In [ ]:
# create a range of coordinates at step 2000 m
stepgrid = 2000 # m
edgesemilength = 8000
xsteps = np.linspace(xs-edgesemilength, xs+edgesemilength, int(2*edgesemilength/stepgrid))
ysteps = np.linspace(ys-edgesemilength, ys+edgesemilength, int(2*edgesemilength/stepgrid))

# create the mesh based on these arrays
eastings, northings = np.meshgrid(xsteps, ysteps)
elevations = eastings*0

ux,uy,uz = okada(eastings, northings, elevations, **params)

In [ ]:
fig = plot3Dfield(ux, uy, uz, eastings, northings, arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90, scalevertfactor = 60000)

print(f'Maximum measured vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# call a function that return the coordinates of the surface representig the Okada source
ox, oy, oz = plotOkada(xs, ys, zs, azimuth, dip, length, width)

# Add the real dike source
fig.add_trace(go.Surface(x=ox, y=oy, z=oz,                         
                         autocolorscale=False,
                    name='estimated source', showscale=False))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field')

fig.show()

## let's invert!

As done previously, we try to invert the synthetic strain field just calculated to see if we find the nominal solution again.

We skip the exact synthetic data step and go directly to the data to which we add noise.

In [ ]:
sigmahorz = 0.01 #m
sigmavert = 0.02 #m

noisy_ux = ux + np.random.normal(0, sigmahorz, ux.shape)
noisy_uy = uy + np.random.normal(0, sigmahorz, uy.shape)
noisy_uz = uz + np.random.normal(0, sigmavert, uz.shape)

Re-define the misfit function for an Okada model

In [ ]:
def wrmse(modelparams, x, y, z, ux, uy, uz, eux=1., euy=1., euz=1., nu=0.25): 
    
    xs, ys, zs, azimuth, dip, length, width, opening = modelparams
    
    # calculate the predicted Okada deformation
    ux_, uy_, uz_ = okada(x, y, z, xs, ys, zs, azimuth, dip, length, width, opening=opening, nu=nu)

    return np.sqrt((((ux_ - ux)/eux) ** 2 + ((uy_ - uy)/euy) ** 2 + ((uz_ - uz)/euz) ** 2).mean())

we now apply the same inversion scheme that we used previously

In [ ]:
# Inversion
# ----------------

# define the misfit for the actual data
errfunc = lambda modelparams: wrmse(modelparams, eastings, northings, elevations, 
                                                             noisy_ux, noisy_uy, noisy_uz, nu=0.25)

# set the constraints for each model parameter
#          xs[m]      ys[m]    zs[m]   azimuth[°]    dip[°]  length[m]  width[m]  opening[m]
lwbound = [xs-10000, ys-10000, -5000,    0,           0,     100,          100,       0]        
upbound = [xs+10000, ys+10000,  -10,      360,          89.,   6000,         6000,      5]

# define an intial guess of the solution
#                xs[m]      ys[m]    zs[m]   azi[°]    dip[°] len[m]  wid[m]  opening[m]
initial_guess = [xs+1000,  ys-1000,     -500,   100,   85,   2000,    3000,     1]

# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening  = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

# calculate the predicted deformation due to the optimal solution found
ux_, uy_, uz_ = okada(eastings, northings, elevations, 
                              optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, 
                              optimal_width, opening=optimal_opening)


comparison = pd.DataFrame([[xs, ys, zs, azimuth, dip, length, width, opening], 
                           [optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, 
                            optimal_length, optimal_width, optimal_opening]], 
                          columns=['easting [m]','northing [m]','-depth [m]','azimuth [°]','dip [°]', 
                                   'length [m]','width [m]','opening [m]'])

comparison.index = ['real','retrieved']

plot the fields and compare the solution with the real parameters

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(ux, uy, uz, eastings, northings, elevations, color='black', arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([ux.ravel(), uy.ravel(), uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Surface(x=ox, y=oy, z=oz,
                         surfacecolor = ox*0,
                         autocolorscale=False,
                    name='real source', showscale=False))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, eastings, northings, elevations, fig=fig, color='red', arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

ox_, oy_, oz_ = plotOkada(optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width)

# Add the real point source
fig.add_trace(go.Surface(x=ox_, y=oy_, z=oz_,
                         surfacecolor = ox*0+3,
                         autocolorscale=False,
                    name='estimated source', showscale=False))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field')

display(comparison)

fig.show()

again, in real cases the number of GNSS-like measurements unfortunately is much lower ...

let's cut some points in the data

In [ ]:
# number of stations we want to save
nstations = 10

idx = random.sample(list(range(np.size(noisy_ux))), nstations)
few_noisy_ux = noisy_ux.ravel()[idx]
few_noisy_uy = noisy_uy.ravel()[idx]
few_noisy_uz = noisy_uz.ravel()[idx]
few_eastings = eastings.ravel()[idx]
few_northings = northings.ravel()[idx]
few_elevations = elevations.ravel()[idx]

same scheme for inversion, but now on smaller dataset

In [ ]:
# Inversion
# ----------------

errfunc = lambda modelparams: wrmse(modelparams, few_eastings, few_northings, few_elevations, 
                                                             few_noisy_ux, few_noisy_uy, few_noisy_uz, nu=0.25)


#          xs[m]      ys[m]    zs[m]   azimuth[°]    dip[°]  length[m]  width[m]  opening[m]
lwbound = [xs-10000, ys-10000, -5000,    0,           0,     100,          100,       0]        
upbound = [xs+10000, ys+10000,  -50,    360,          89.,   5000,         5000,      5]

#                xs[m]      ys[m]    zs[m]   azi[°]    dip[°] len[m]  wid[m]  opening[m]
initial_guess = [xs+1000,  ys-1000,     -500,   100,   85,   4000,    3000,     2]

# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening  = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

ux_, uy_, uz_ = okada(few_eastings, few_northings, few_elevations, 
                              optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, 
                              optimal_width, opening=optimal_opening)


comparison = pd.DataFrame([[xs, ys, zs, azimuth, dip, length, width, opening], 
                           [optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening]], 
                          columns=['easting [m]','northing [m]','-depth [m]','azimuth [°]','dip [°]', 'length [m]','width [m]','opening [m]'])
comparison.index = ['real','retrieved']


plot the results and compare the parameters

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(few_noisy_ux, few_noisy_uy, few_noisy_uz, few_eastings, few_northings, few_elevations, 
                  color='black', arrow_tip_ratio = 0.2,
                  arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([few_noisy_ux.ravel(), few_noisy_uy.ravel(), few_noisy_uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Surface(x=ox, y=oy, z=oz,
                         surfacecolor = ox*0,
                         autocolorscale=False,
                    name='real source', showscale=False))

# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, few_eastings, few_northings, few_elevations, fig=fig, color='red', arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90)

ox_, oy_, oz_ = plotOkada(optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Surface(x=ox_, y=oy_, z=oz_,
                         surfacecolor = ox*0+3,
                         autocolorscale=False,
                    name='estimated source', showscale=False))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field'
)

display(comparison)

fig.show()

 finally, now we only need to introduce uncertainty about the data

In [ ]:
# create false uncertainties but realistic as deviation from the nominal data

eux = np.abs((few_noisy_ux - ux.ravel()[idx]))
euy = np.abs((few_noisy_uy - uy.ravel()[idx]))
euz = np.abs((few_noisy_uz - uz.ravel()[idx]))

 is it getting boring? we have to do the inversion again

In [ ]:
# Inversion
# ----------------
errfunc = lambda modelparams: wrmse(modelparams, few_eastings, few_northings, few_elevations, 
                                                            few_noisy_ux, few_noisy_uy, few_noisy_uz,
                                                            eux=eux, euy=euy, euz=euz, nu=0.25)

#          xs[m]      ys[m]    zs[m]   azimuth[°]    dip[°]  length[m]  width[m]  opening[m]
lwbound = [xs-10000, ys-10000, -5000,    0,           0,     100,          100,       0]        
upbound = [xs+10000, ys+10000,  -50,    360,          89.,   5000,         5000,      5]

#                xs[m]      ys[m]    zs[m]   azi[°]    dip[°] len[m]  wid[m]  opening[m]
initial_guess = [xs+1000,  ys-1000,     -500,   100,   85,   4000,    3000,     2]


# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening  = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

ux_, uy_, uz_ = okada(few_eastings, few_northings, few_elevations, 
                              optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, 
                              optimal_width, opening=optimal_opening)


comparison = pd.DataFrame([[xs, ys, zs, azimuth, dip, length, width, opening], 
                           [optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening]], 
                          columns=['easting [m]','northing [m]','-depth [m]','azimuth [°]','dip [°]', 'length [m]','width [m]','opening [m]'])
comparison.index = ['real','retrieved']

and plot + compare the results

In [ ]:
# Plot Real deformation field
fig = plot3Dfield(few_noisy_ux, few_noisy_uy, few_noisy_uz, few_eastings, few_northings, few_elevations, 
                  color='black', arrow_tip_ratio = 0.2,
                  arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([few_noisy_ux.ravel(), few_noisy_uy.ravel(), few_noisy_uz.ravel()], axis=0)):0.3f} m')

# Add the real point source
fig.add_trace(go.Surface(x=ox, y=oy, z=oz,
                         surfacecolor = ox*0,
                         autocolorscale=False,
                    name='real source', showscale=False))


# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, few_eastings, few_northings, few_elevations, fig=fig, color='red', arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

ox_, oy_, oz_ = plotOkada(optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width)

# Add the real point source
fig.add_trace(go.Surface(x=ox_, y=oy_, z=oz_,
                         surfacecolor = ox*0+3,
                         autocolorscale=False,
                    name='estimated source', showscale=False))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field')

display(comparison)

fig.show()

# We are done!

Thank you 

# INVERSION SCHEME



In [ ]:
# Inversion
# ----------------

# for all the stations define locations and measurements:

station_eastings = np.array([100, 200, 300, 400])
station_northings = np.array([100, 200, 300, 400])
station_elevations = np.array([100, 200, 300, 400])

station_ux = np.array([0.1, 0.2, 0.3, 0.4])
station_uy = np.array([0.1, 0.2, 0.3, 0.4])
station_uz = np.array([0.1, 0.2, 0.3, 0.4])

eux = np.array([0.1, 0.2, 0.3, 0.4])
euy = np.array([0.1, 0.2, 0.3, 0.4])
euz = np.array([0.1, 0.2, 0.3, 0.4])


# define the misfit function DEPENDING ON THE MODEL SOURCE!!!
errfunc = lambda modelparams: wrmse(modelparams, station_eastings, station_northings, station_elevations, 
                                                            station_ux, station_uy, station_uz,
                                                            eux=eux, euy=euy, euz=euz, nu=0.25)

# define the constraints DEPENDING ON THE MODEL SOURCE!!!
#          xs[m]      ys[m]    zs[m]   azimuth[°]    dip[°]  length[m]  width[m]  opening[m]
lwbound = [xs-10000, ys-10000, -5000,    0,           0,     100,          100,       0]        
upbound = [xs+10000, ys+10000,  -50,    360,          89.,   5000,         5000,      5]

# define the first gues of the parameters DEPENDING ON THE MODEL SOURCE!!!
#                xs[m]      ys[m]    zs[m]   azi[°]    dip[°] len[m]  wid[m]  opening[m]
initial_guess = [xs+1000,  ys-1000,     -500,   100,   85,   4000,    3000,     2]


# Global heuristic optimization, 2-steps: global search + direct search
first_guess = direct(errfunc, bounds=list(zip(lwbound, upbound)))

# refinining two solutions
opt1 = minimize(errfunc, first_guess.x, method='nelder-mead', bounds=list(zip(lwbound, upbound)))
opt2 = minimize(errfunc, initial_guess, method='nelder-mead', bounds=list(zip(lwbound, upbound)))

# choose the best result
opt = opt1 if opt1.fun<opt2.fun else opt2

optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening  = opt.x

print(f'Found WRMSE Global minimum {errfunc(opt.x)}')

# calculate the model prediction DEPENDING ON THE MODEL SOURCE!!!
ux_, uy_, uz_ = okada(few_eastings, few_northings, few_elevations, 
                              optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, 
                              optimal_width, opening=optimal_opening)


comparison = pd.DataFrame([[xs, ys, zs, azimuth, dip, length, width, opening], 
                           [optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width, optimal_opening]], 
                          columns=['easting [m]','northing [m]','-depth [m]','azimuth [°]','dip [°]', 'length [m]','width [m]','opening [m]'])
comparison.index = ['real','retrieved']


# Plot Real deformation field
fig = plot3Dfield(station_ux, station_uy, station_uz, station_eastings, station_northings, station_elevations, 
                  color='black', arrow_tip_ratio = 0.2,
                  arrow_starting_ratio = 0.90)

print(f'Maximum measured vector length {np.max(np.linalg.norm([station_ux.ravel(), station_uy.ravel(), station_uz.ravel()], axis=0)):0.3f} m')

# Add the real point source DEPENDING ON THE MODEL SOURCE!!!
fig.add_trace(go.Surface(x=ox, y=oy, z=oz,
                         surfacecolor = ox*0,
                         autocolorscale=False,
                    name='real source', showscale=False))


# Plot the model-predicted deformation field
fig = plot3Dfield(ux_, uy_, uz_, station_eastings, station_northings, station_elevations, fig=fig, color='red', arrow_tip_ratio = 0.2,
                arrow_starting_ratio = 0.90)

print(f'Maximum predicted vector length {np.max(np.linalg.norm([ux_.ravel(), uy_.ravel(), uz_.ravel()], axis=0)):0.3f} m')

# plot the source 
ox_, oy_, oz_ = plotOkada(optimal_xs, optimal_ys, optimal_zs, optimal_azimuth, optimal_dip, optimal_length, optimal_width)

# Add the real point source DEPENDING ON THE MODEL SOURCE!!!
fig.add_trace(go.Surface(x=ox_, y=oy_, z=oz_,
                         surfacecolor = ox*0+3,
                         autocolorscale=False,
                    name='estimated source', showscale=False))

camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=1.5))

fig.update_layout(
    width=900, height=900, scene_camera=camera, title='Deformation Field')

display(comparison)

fig.show()